In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import resample
import datetime as dt
from math import ceil, sqrt
from Utils import *
pd.set_option('max_colwidth', None)
from EvaluationMetric import *
from xgboost import XGBClassifier

In [11]:
def make_mi_scores(X, y):
    mi_scores = mutual_info_classif(X, y)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

Put the data here

In [4]:
def_feature = pd.read_csv("input/Xente_Variable_Definitions.csv")
raw_data = pd.read_csv("input/training.csv")
X_test = pd.read_csv("input/test.csv")
sample_submission = pd.read_csv("input/sample_submission.csv")

#attribut initialization
cols_unique_value = [] #Will be droped
for col in raw_data.columns : 
    if len(raw_data[col].unique()) == 1 :
        cols_unique_value.append(col)

medium_cardianlity_cols = ["ProductId"]

#Data transformation
raw_data['TransactionStartTime'] = pd.to_datetime(raw_data['TransactionStartTime'])
X_test['TransactionStartTime'] = pd.to_datetime(X_test['TransactionStartTime'])

#Data cleaning
data = raw_data.copy()
data = data.dropna(axis=0) #Drop observations/rows with missing values
X_test=X_test.dropna(axis=0)
data.drop(cols_unique_value, axis=1, inplace=True)
X_test.drop(cols_unique_value, axis=1, inplace=True)

#Prepare index for the submission
index_val = list(X_test.TransactionId.values.tolist())

#Set the df index to the Transction Id
data = transactioId_to_index(data)
X_test = transactioId_to_index(X_test)

#Adding data
data = adding_date_col(data, 'TransactionStartTime')
X_test = adding_date_col(X_test, 'TransactionStartTime')


# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in data.columns if data[cname].nunique() < 15 and 
                        data[cname].dtype == "object"]
low_cardinality_cols.append('PricingStrategy')

#transform columns BatchId_54 to number

l_col_str = ["BatchId", "AccountId", "SubscriptionId", "CustomerId", "ProviderId", "ProductId", "ChannelId"]
for col in l_col_str:
    data[['dc', 'new_col']] = data[col].str.split("_", expand = True)
    data.drop(['dc',col], inplace=True, axis=1)
    data.rename(columns={"new_col": col}, inplace=True)
    data[col] = data[col].astype('int')
    X_test[['dc', 'new_col']] = X_test[col].str.split("_", expand = True)
    X_test.drop(['dc',col], inplace=True, axis=1)
    X_test.rename(columns={"new_col": col}, inplace=True)
    X_test[col] = X_test[col].astype('int')
data['PricingStrategy'] = data['PricingStrategy'].astype('int')
X_test['PricingStrategy'] = X_test['PricingStrategy'].astype('int')
data['Value'] = data['Value'].astype('float')
X_test['Value'] = X_test['Value'].astype('float')

X_test['PricingStrategy'] = X_test['PricingStrategy'].astype('str')
data['PricingStrategy'] = data['PricingStrategy'].astype('str')

#Data splitting
y = data.FraudResult #The target label
X = data.copy()
X.drop(['FraudResult'], axis=1, inplace=True) #Only the features data
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)

cat_cols = [col for col in X.columns if X[col].dtype == "object"]#liste of obejct columns
cat_cols.append("PricingStrategy")#pcq mm si c'est un chiffre il faut le considérer comme une catégorie

#print(describe)


#low_cardinality_cols=["ProviderId", "ProductCategory", "ChannelId", "PricingStrategy"] 
#train_X[low_cardinality_cols] = train_X[low_cardinality_cols].astype(str) 
#val_X[low_cardinality_cols] = val_X[low_cardinality_cols].astype(str) 

OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(train_X[low_cardinality_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(val_X[low_cardinality_cols]))
OH_cols_test = pd.DataFrame(OH_encoder.transform(X_test[low_cardinality_cols]))
OH_cols_train.columns = OH_encoder.get_feature_names_out()
OH_cols_valid.columns = OH_encoder.get_feature_names_out()
OH_cols_test.columns = OH_encoder.get_feature_names_out()

# One-hot encoding removed index; put it back
OH_cols_train.index = train_X.index
OH_cols_valid.index = val_X.index
OH_cols_test.index = X_test.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = train_X.drop(low_cardinality_cols, axis=1)
num_X_valid = val_X.drop(low_cardinality_cols, axis=1)
num_X_test = X_test.drop(low_cardinality_cols, axis=1)

OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)
OH_X_test = pd.concat([num_X_test, OH_cols_test], axis=1)

# Problème de string
OH_X_train.drop(['TransactionStartTime'], inplace=True, axis=1)
OH_X_valid.drop(['TransactionStartTime'], inplace=True, axis=1)
OH_X_test.drop(['TransactionStartTime'], inplace=True, axis=1)
OH_X_train.columns = OH_X_train.columns.astype(str)
OH_X_valid.columns = OH_X_valid.columns.astype(str)
OH_X_test.columns = OH_X_test.columns.astype(str)

train_X = OH_X_train
val_X = OH_X_valid
test_X = OH_X_test
X_entier = pd.concat([train_X, val_X])

In [8]:
#descriptiontous = 'En suprimant les données avec le plus petit MI'
descriptiontous = 'La base'

res = []

model1 = DecisionTreeClassifier(max_leaf_nodes= 6, random_state=1)
model1.fit(train_X, train_y)
preds_val1 = model1.predict(val_X)
model_test1 = model1.predict(test_X)
description = descriptiontous+"max_leaf_nodes : 6 + with date columns"
metrics1 = listmetrics(val_y, preds_val1, "Decision Tree Classifier", description)
m1Public = [0.666666666]
m1Private = [0.661016949]
res.append(metrics1)
df1 = getscoreforcsv(index_val, model_test1, name_file = "TreeClass5leaf.csv")


model2 = RandomForestClassifier(n_estimators=36, criterion = 'entropy', random_state=1)
model2.fit(train_X, train_y)
preds_val2 = model2.predict(val_X)
model_test2 = model2.predict(test_X)
description = descriptiontous+"n_estimators : 36 + Entropy"
metrics2 = listmetrics(val_y, preds_val2, "Random Forest Classifier", description)
m2Public = [0.690909090]
m2Private = [0.649122807]
res.append(metrics2)
df2 = getscoreforcsv(index_val, model_test2, name_file = "RandomForestN36Entropy.csv")

model2 = XGBClassifier(scale_pos_weight=22, random_state=1)
model2.fit(train_X, train_y)
preds_val2 = model2.predict(val_X)
model_test2 = model2.predict(test_X)
description = descriptiontous+"scale_pos_weight = 22"
metrics2 = listmetrics(val_y, preds_val2, "XGBClassifier", description)
m3Public = [0.677419354]
m3Private = [0.676923076]
res.append(metrics2)
df2 = getscoreforcsv(index_val, model_test2, name_file = "XGBClassifierW22.csv")

#oversamplesimple
X_con = pd.concat([train_X, train_y], axis=1) 
not_fraud = X_con[X_con.FraudResult==0]
fraud = X_con[X_con.FraudResult==1]
fraud_upsampled = resample(fraud, replace=True, n_samples=len(not_fraud), # match number in majority class
                          random_state=1) # reproducible results
upsampled = pd.concat([not_fraud, fraud_upsampled])
train_y_over_sampled = upsampled.FraudResult
train_X_over_sampled = upsampled.drop('FraudResult', axis=1)
upsampledmodel = RandomForestClassifier(n_estimators=36, criterion = 'entropy', random_state=1)
upsampledmodel.fit(train_X_over_sampled, train_y_over_sampled)
upsampled_pred = upsampledmodel.predict(val_X)
model_test = upsampledmodel.predict(OH_X_test)
description = descriptiontous+"upsampled, n_estimators=36, criterion = entropy"
metrics = listmetrics(val_y, upsampled_pred, "RandomForestClassifierUpperSampling", description)
m4Public = [0.678571428]
m4Private = [0.637168141]
res.append(metrics)
df = getscoreforcsv(index_val, model_test, name_file = "RandomForestClassifierUpSample.csv")

#Undersampling
not_fraud_downsampled  = resample(not_fraud, replace=False, n_samples=len(fraud), # match number in minority  class
                          random_state=1) # reproducible results
downsampled  = pd.concat([not_fraud_downsampled , fraud])
train_y_undersampled = downsampled.FraudResult
train_X_undersampled = downsampled.drop('FraudResult', axis=1)
undersampled = RandomForestClassifier(n_estimators=36, criterion = 'entropy', random_state=1)
undersampled.fit(train_X_over_sampled, train_y_over_sampled)
undersampled_pred = undersampled.predict(val_X)
model_test = undersampled.predict(OH_X_test)
description = descriptiontous+"undersampled, n_estimators=36, criterion = entropy"
metrics = listmetrics(val_y, undersampled_pred, "RandomForestClassifierUnderSampling", description)
m5Public = [0.549019607]
m5Private = [0.480769230]
res.append(metrics)
df = getscoreforcsv(index_val, model_test, name_file = "RandomForestClassifierUndersampled.csv")


dfres = listmetricsintodf(res)
dfres["OldPublicScore"] = [m1Public, m2Public, m3Public, m4Public, m5Public]
dfres["OldPrivateScore"] = [m1Private, m2Private, m3Private, m4Private, m5Private]
dfres["MeanOurMetrics"] = dfres[['Precision', 'Recall','F1-score','Mcc']].mean(axis=1)
dfres.to_csv('output/resultatsTestEverything.csv', index=False) 
dfres

done
done
done
done
done


,Model,Description,Date,Precision,Recall,F1-score,LogLoss,Mcc,PublicScore,PrivateScore,OldPublicScore,OldPrivateScore,MeanOurMetrics
0,Decision Tree Classifier,La basemax_leaf_nodes : 6 + with date columns,2023-05-06 20:37:01.701978,0.891892,0.825,0.857143,0.016578,0.857566,NaN,NaN,[0.666666666],[0.661016949],0.857900
1,Random Forest Classifier,La basen_estimators : 36 + Entropy,2023-05-06 20:37:02.891559,0.800000,0.900,0.847059,0.019592,0.848262,NaN,NaN,[0.69090909],[0.649122807],0.848830
2,XGBClassifier,La basescale_pos_weight = 22,2023-05-06 20:37:04.316049,0.840909,0.925,0.880952,0.015071,0.881747,NaN,NaN,[0.677419354],[0.676923076],0.882152
3,RandomForestClassifierUpperSampling,"La baseupsampled, n_estimators=36, criterion = entropy",2023-05-06 20:37:06.774426,0.765957,0.900,0.827586,0.022606,0.829975,NaN,NaN,[0.678571428],[0.637168141],0.830880
4,RandomForestClassifierUnderSampling,"La baseundersampled, n_estimators=36, criterion = entropy",2023-05-06 20:37:09.025865,0.765957,0.900,0.827586,0.022606,0.829975,NaN,NaN,[0.549019607],[0.48076923],0.830880


In [39]:
# Supprimer les MI trop faible
def_feature = pd.read_csv("input/Xente_Variable_Definitions.csv")
raw_data = pd.read_csv("input/training.csv")
X_test = pd.read_csv("input/test.csv")
sample_submission = pd.read_csv("input/sample_submission.csv")

#attribut initialization
cols_unique_value = [] #Will be droped
for col in raw_data.columns : 
    if len(raw_data[col].unique()) == 1 :
        cols_unique_value.append(col)

medium_cardianlity_cols = ["ProductId"]

#Data transformation
raw_data['TransactionStartTime'] = pd.to_datetime(raw_data['TransactionStartTime'])
X_test['TransactionStartTime'] = pd.to_datetime(X_test['TransactionStartTime'])

#Data cleaning
data = raw_data.copy()
data = data.dropna(axis=0) #Drop observations/rows with missing values
X_test=X_test.dropna(axis=0)
data.drop(cols_unique_value, axis=1, inplace=True)
X_test.drop(cols_unique_value, axis=1, inplace=True)

#Prepare index for the submission
index_val = list(X_test.TransactionId.values.tolist())

#Set the df index to the Transction Id
data = transactioId_to_index(data)
X_test = transactioId_to_index(X_test)

#Adding data
data = adding_date_col(data, 'TransactionStartTime')
X_test = adding_date_col(X_test, 'TransactionStartTime')


# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in data.columns if data[cname].nunique() < 15 and 
                        data[cname].dtype == "object"]
low_cardinality_cols.append('PricingStrategy')

#transform columns BatchId_54 to number

l_col_str = ["BatchId", "AccountId", "SubscriptionId", "CustomerId", "ProviderId", "ProductId", "ChannelId"]
for col in l_col_str:
    data[['dc', 'new_col']] = data[col].str.split("_", expand = True)
    data.drop(['dc',col], inplace=True, axis=1)
    data.rename(columns={"new_col": col}, inplace=True)
    data[col] = data[col].astype('int')
    X_test[['dc', 'new_col']] = X_test[col].str.split("_", expand = True)
    X_test.drop(['dc',col], inplace=True, axis=1)
    X_test.rename(columns={"new_col": col}, inplace=True)
    X_test[col] = X_test[col].astype('int')
data['PricingStrategy'] = data['PricingStrategy'].astype('int')
X_test['PricingStrategy'] = X_test['PricingStrategy'].astype('int')
data['Value'] = data['Value'].astype('float')
X_test['Value'] = X_test['Value'].astype('float')

X_test['PricingStrategy'] = X_test['PricingStrategy'].astype('str')
data['PricingStrategy'] = data['PricingStrategy'].astype('str')

#Data splitting
y = data.FraudResult #The target label
X = data.copy()
X.drop(['FraudResult'], axis=1, inplace=True) #Only the features data
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)

cat_cols = [col for col in X.columns if X[col].dtype == "object"]#liste of obejct columns
cat_cols.append("PricingStrategy")#pcq mm si c'est un chiffre il faut le considérer comme une catégorie

#print(describe)


#low_cardinality_cols=["ProviderId", "ProductCategory", "ChannelId", "PricingStrategy"] 
#train_X[low_cardinality_cols] = train_X[low_cardinality_cols].astype(str) 
#val_X[low_cardinality_cols] = val_X[low_cardinality_cols].astype(str) 

OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(train_X[low_cardinality_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(val_X[low_cardinality_cols]))
OH_cols_test = pd.DataFrame(OH_encoder.transform(X_test[low_cardinality_cols]))
OH_cols_train.columns = OH_encoder.get_feature_names_out()
OH_cols_valid.columns = OH_encoder.get_feature_names_out()
OH_cols_test.columns = OH_encoder.get_feature_names_out()

# One-hot encoding removed index; put it back
OH_cols_train.index = train_X.index
OH_cols_valid.index = val_X.index
OH_cols_test.index = X_test.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = train_X.drop(low_cardinality_cols, axis=1)
num_X_valid = val_X.drop(low_cardinality_cols, axis=1)
num_X_test = X_test.drop(low_cardinality_cols, axis=1)

OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)
OH_X_test = pd.concat([num_X_test, OH_cols_test], axis=1)

# Problème de string
OH_X_train.drop(['TransactionStartTime'], inplace=True, axis=1)
OH_X_valid.drop(['TransactionStartTime'], inplace=True, axis=1)
OH_X_test.drop(['TransactionStartTime'], inplace=True, axis=1)
OH_X_train.columns = OH_X_train.columns.astype(str)
OH_X_valid.columns = OH_X_valid.columns.astype(str)
OH_X_test.columns = OH_X_test.columns.astype(str)

train_X = OH_X_train
val_X = OH_X_valid
test_X = OH_X_test
X_entier = pd.concat([train_X, val_X])

mi_scores = make_mi_scores(X_entier , y)

verylowMI = list(mi_scores[mi_scores<0.0001].index)
lowMI = list(mi_scores[(mi_scores>=0.0001)&(mi_scores<0.001)].index)
mediumMI = list(mi_scores[(mi_scores>=0.001)&(mi_scores<0.01)].index)
highMI = list(mi_scores[mi_scores>=0.01].index)

In [40]:
descriptiontous = 'Sans veryLow MI'

train_X.drop(verylowMI, inplace=True, axis=1)
val_X.drop(verylowMI, inplace=True, axis=1)
test_X.drop(verylowMI, inplace=True, axis=1)
X_entier.drop(verylowMI, inplace=True, axis=1)
res = []

model1 = DecisionTreeClassifier(max_leaf_nodes= 6, random_state=1)
model1.fit(train_X, train_y)
preds_val1 = model1.predict(val_X)
model_test1 = model1.predict(test_X)
description = descriptiontous+"max_leaf_nodes : 6 + with date columns"
metrics1 = listmetrics(val_y, preds_val1, "Decision Tree Classifier", description)
m1Public = [0.666666666, 0.666666666]
m1Private = [0.661016949, 0.661016949]
res.append(metrics1)
df1 = getscoreforcsv(index_val, model_test1, name_file = "TreeClass5leaf.csv")


model2 = RandomForestClassifier(n_estimators=36, criterion = 'entropy', random_state=1)
model2.fit(train_X, train_y)
preds_val2 = model2.predict(val_X)
model_test2 = model2.predict(test_X)
description = descriptiontous+"n_estimators : 36 + Entropy"
metrics2 = listmetrics(val_y, preds_val2, "Random Forest Classifier", description)
m2Public = [0.690909090, 0.666666666]
m2Private = [0.649122807, 0.649122807
]
res.append(metrics2)
df2 = getscoreforcsv(index_val, model_test2, name_file = "RandomForestN36Entropy.csv")

##################ICi donne un 0.71 de scrore !!! mtn comprendre pq en diminuant les feature si bon score mais que pour lui !############

model2 = XGBClassifier(scale_pos_weight=22, random_state=1)
model2.fit(train_X, train_y)
preds_val2 = model2.predict(val_X)
model_test2 = model2.predict(test_X)
description = descriptiontous+"scale_pos_weight = 22"
metrics2 = listmetrics(val_y, preds_val2, "XGBClassifier", description)
m3Public = [0.677419354, 0.711864406]
m3Private = [0.676923076,  0.715447154]
res.append(metrics2)
df2 = getscoreforcsv(index_val, model_test2, name_file = "XGBClassifierW22.csv")

#oversamplesimple
X_con = pd.concat([train_X, train_y], axis=1) 
not_fraud = X_con[X_con.FraudResult==0]
fraud = X_con[X_con.FraudResult==1]
fraud_upsampled = resample(fraud, replace=True, n_samples=len(not_fraud), # match number in majority class
                          random_state=1) # reproducible results
upsampled = pd.concat([not_fraud, fraud_upsampled])
train_y_over_sampled = upsampled.FraudResult
train_X_over_sampled = upsampled.drop('FraudResult', axis=1)
upsampledmodel = RandomForestClassifier(n_estimators=36, criterion = 'entropy', random_state=1)
upsampledmodel.fit(train_X_over_sampled, train_y_over_sampled)
upsampled_pred = upsampledmodel.predict(val_X)
model_test = upsampledmodel.predict(OH_X_test)
description = descriptiontous+"upsampled, n_estimators=36, criterion = entropy"
metrics = listmetrics(val_y, upsampled_pred, "RandomForestClassifierUpperSampling", description)
m4Public = [0.678571428, 0.642857142]
m4Private = [0.637168141, 0.620689655]
res.append(metrics)
df = getscoreforcsv(index_val, model_test, name_file = "RandomForestClassifierUpSample.csv")

#Undersampling
not_fraud_downsampled  = resample(not_fraud, replace=False, n_samples=len(fraud), # match number in minority  class
                          random_state=1) # reproducible results
downsampled  = pd.concat([not_fraud_downsampled , fraud])
train_y_undersampled = downsampled.FraudResult
train_X_undersampled = downsampled.drop('FraudResult', axis=1)
undersampled = RandomForestClassifier(n_estimators=36, criterion = 'entropy', random_state=1)
undersampled.fit(train_X_over_sampled, train_y_over_sampled)
undersampled_pred = undersampled.predict(val_X)
model_test = undersampled.predict(OH_X_test)
description = descriptiontous+"undersampled, n_estimators=36, criterion = entropy"
metrics = listmetrics(val_y, undersampled_pred, "RandomForestClassifierUnderSampling", description)
m5Public = [0.549019607, 0.642857142]
m5Private = [0.480769230, 0.620689655]
res.append(metrics)
df = getscoreforcsv(index_val, model_test, name_file = "RandomForestClassifierUndersampled.csv")


dfres = listmetricsintodf(res)
dfres["OldPublicScore"] = [m1Public, m2Public, m3Public, m4Public, m5Public]
dfres["OldPrivateScore"] = [m1Private, m2Private, m3Private, m4Private, m5Private]
dfres["MeanOurMetrics"] = dfres[['Precision', 'Recall','F1-score','Mcc']].mean(axis=1)
dfres.to_csv('output/resultatsTestEverything.csv', mode='a',index=False,header = False) 
dfres

done
done
done
done
done


,Model,Description,Date,Precision,Recall,F1-score,LogLoss,Mcc,PublicScore,PrivateScore,OldPublicScore,OldPrivateScore,MeanOurMetrics
0,Decision Tree Classifier,Sans veryLow MImax_leaf_nodes : 6 + with date columns,2023-05-06 21:06:44.038023,0.942857,0.825,0.880000,0.013564,0.881780,NaN,NaN,[0.666666666],[0.661016949],0.882409
1,Random Forest Classifier,Sans veryLow MIn_estimators : 36 + Entropy,2023-05-06 21:06:45.137578,0.853659,0.875,0.864198,0.016578,0.864033,NaN,NaN,[0.69090909],[0.649122807],0.864222
2,XGBClassifier,Sans veryLow MIscale_pos_weight = 22,2023-05-06 21:06:46.430420,0.837209,0.900,0.867470,0.016578,0.867809,NaN,NaN,[0.677419354],[0.676923076],0.868122
3,RandomForestClassifierUpperSampling,"Sans veryLow MIupsampled, n_estimators=36, criterion = entropy",2023-05-06 21:06:48.589144,0.818182,0.900,0.857143,0.018085,0.857869,NaN,NaN,[0.678571428],[0.637168141],0.858298
4,RandomForestClassifierUnderSampling,"Sans veryLow MIundersampled, n_estimators=36, criterion = entropy",2023-05-06 21:06:50.632936,0.818182,0.900,0.857143,0.018085,0.857869,NaN,NaN,[0.549019607],[0.48076923],0.858298


In [14]:
descriptiontous = 'Sans Low MI'

train_X.drop(lowMI, inplace=True, axis=1)
val_X.drop(lowMI, inplace=True, axis=1)
test_X.drop(lowMI, inplace=True, axis=1)
X_entier.drop(lowMI, inplace=True, axis=1)
res = []

model1 = DecisionTreeClassifier(max_leaf_nodes= 6, random_state=1)
model1.fit(train_X, train_y)
preds_val1 = model1.predict(val_X)
model_test1 = model1.predict(test_X)
description = descriptiontous+"max_leaf_nodes : 6 + with date columns"
metrics1 = listmetrics(val_y, preds_val1, "Decision Tree Classifier", description)
m1Public = [0.666666666]
m1Private = [0.661016949]
res.append(metrics1)
df1 = getscoreforcsv(index_val, model_test1, name_file = "TreeClass5leaf.csv")


model2 = RandomForestClassifier(n_estimators=36, criterion = 'entropy', random_state=1)
model2.fit(train_X, train_y)
preds_val2 = model2.predict(val_X)
model_test2 = model2.predict(test_X)
description = descriptiontous+"n_estimators : 36 + Entropy"
metrics2 = listmetrics(val_y, preds_val2, "Random Forest Classifier", description)
m2Public = [0.690909090]
m2Private = [0.649122807]
res.append(metrics2)
df2 = getscoreforcsv(index_val, model_test2, name_file = "RandomForestN36Entropy.csv")

model2 = XGBClassifier(scale_pos_weight=22, random_state=1)
model2.fit(train_X, train_y)
preds_val2 = model2.predict(val_X)
model_test2 = model2.predict(test_X)
description = descriptiontous+"scale_pos_weight = 22"
metrics2 = listmetrics(val_y, preds_val2, "XGBClassifier", description)
m3Public = [0.677419354]
m3Private = [0.676923076]
res.append(metrics2)
df2 = getscoreforcsv(index_val, model_test2, name_file = "XGBClassifierW22.csv")

#oversamplesimple
X_con = pd.concat([train_X, train_y], axis=1) 
not_fraud = X_con[X_con.FraudResult==0]
fraud = X_con[X_con.FraudResult==1]
fraud_upsampled = resample(fraud, replace=True, n_samples=len(not_fraud), # match number in majority class
                          random_state=1) # reproducible results
upsampled = pd.concat([not_fraud, fraud_upsampled])
train_y_over_sampled = upsampled.FraudResult
train_X_over_sampled = upsampled.drop('FraudResult', axis=1)
upsampledmodel = RandomForestClassifier(n_estimators=36, criterion = 'entropy', random_state=1)
upsampledmodel.fit(train_X_over_sampled, train_y_over_sampled)
upsampled_pred = upsampledmodel.predict(val_X)
model_test = upsampledmodel.predict(OH_X_test)
description = descriptiontous+"upsampled, n_estimators=36, criterion = entropy"
metrics = listmetrics(val_y, upsampled_pred, "RandomForestClassifierUpperSampling", description)
m4Public = [0.678571428]
m4Private = [0.637168141]
res.append(metrics)
df = getscoreforcsv(index_val, model_test, name_file = "RandomForestClassifierUpSample.csv")

#Undersampling
not_fraud_downsampled  = resample(not_fraud, replace=False, n_samples=len(fraud), # match number in minority  class
                          random_state=1) # reproducible results
downsampled  = pd.concat([not_fraud_downsampled , fraud])
train_y_undersampled = downsampled.FraudResult
train_X_undersampled = downsampled.drop('FraudResult', axis=1)
undersampled = RandomForestClassifier(n_estimators=36, criterion = 'entropy', random_state=1)
undersampled.fit(train_X_over_sampled, train_y_over_sampled)
undersampled_pred = undersampled.predict(val_X)
model_test = undersampled.predict(OH_X_test)
description = descriptiontous+"undersampled, n_estimators=36, criterion = entropy"
metrics = listmetrics(val_y, undersampled_pred, "RandomForestClassifierUnderSampling", description)
m5Public = [0.549019607]
m5Private = [0.480769230]
res.append(metrics)
df = getscoreforcsv(index_val, model_test, name_file = "RandomForestClassifierUndersampled.csv")


dfres = listmetricsintodf(res)
dfres["OldPublicScore"] = [m1Public, m2Public, m3Public, m4Public, m5Public]
dfres["OldPrivateScore"] = [m1Private, m2Private, m3Private, m4Private, m5Private]
dfres["MeanOurMetrics"] = dfres[['Precision', 'Recall','F1-score','Mcc']].mean(axis=1)
dfres.to_csv('output/resultatsTestEverything.csv', mode='a',index=False,header = False) 
dfres

done
done
done
done
done


,Model,Description,Date,Precision,Recall,F1-score,LogLoss,Mcc,PublicScore,PrivateScore,OldPublicScore,OldPrivateScore,MeanOurMetrics
0,Decision Tree Classifier,Sans Low MImax_leaf_nodes : 6 + with date columns,2023-05-06 20:51:23.078874,0.500000,0.050,0.090909,0.060284,0.157730,NaN,NaN,[0.666666666],[0.661016949],0.199660
1,Random Forest Classifier,Sans Low MIn_estimators : 36 + Entropy,2023-05-06 20:51:24.053805,0.566667,0.425,0.485714,0.054255,0.490016,NaN,NaN,[0.69090909],[0.649122807],0.491849
2,XGBClassifier,Sans Low MIscale_pos_weight = 22,2023-05-06 20:51:24.981709,0.397059,0.675,0.500000,0.081383,0.516687,NaN,NaN,[0.677419354],[0.676923076],0.522187
3,RandomForestClassifierUpperSampling,"Sans Low MIupsampled, n_estimators=36, criterion = entropy",2023-05-06 20:51:27.254243,0.342857,0.600,0.436364,0.093440,0.452385,NaN,NaN,[0.678571428],[0.637168141],0.457902
4,RandomForestClassifierUnderSampling,"Sans Low MIundersampled, n_estimators=36, criterion = entropy",2023-05-06 20:51:29.419018,0.342857,0.600,0.436364,0.093440,0.452385,NaN,NaN,[0.549019607],[0.48076923],0.457902


In [15]:
descriptiontous = 'Sans mediumMI MI'

train_X.drop(mediumMI, inplace=True, axis=1)
val_X.drop(mediumMI, inplace=True, axis=1)
test_X.drop(mediumMI, inplace=True, axis=1)
X_entier.drop(mediumMI, inplace=True, axis=1)
res = []

model1 = DecisionTreeClassifier(max_leaf_nodes= 6, random_state=1)
model1.fit(train_X, train_y)
preds_val1 = model1.predict(val_X)
model_test1 = model1.predict(test_X)
description = descriptiontous+"max_leaf_nodes : 6 + with date columns"
metrics1 = listmetrics(val_y, preds_val1, "Decision Tree Classifier", description)
m1Public = [0.666666666]
m1Private = [0.661016949]
res.append(metrics1)
df1 = getscoreforcsv(index_val, model_test1, name_file = "TreeClass5leaf.csv")


model2 = RandomForestClassifier(n_estimators=36, criterion = 'entropy', random_state=1)
model2.fit(train_X, train_y)
preds_val2 = model2.predict(val_X)
model_test2 = model2.predict(test_X)
description = descriptiontous+"n_estimators : 36 + Entropy"
metrics2 = listmetrics(val_y, preds_val2, "Random Forest Classifier", description)
m2Public = [0.690909090]
m2Private = [0.649122807]
res.append(metrics2)
df2 = getscoreforcsv(index_val, model_test2, name_file = "RandomForestN36Entropy.csv")

model2 = XGBClassifier(scale_pos_weight=22, random_state=1)
model2.fit(train_X, train_y)
preds_val2 = model2.predict(val_X)
model_test2 = model2.predict(test_X)
description = descriptiontous+"scale_pos_weight = 22"
metrics2 = listmetrics(val_y, preds_val2, "XGBClassifier", description)
m3Public = [0.677419354]
m3Private = [0.676923076]
res.append(metrics2)
df2 = getscoreforcsv(index_val, model_test2, name_file = "XGBClassifierW22.csv")

#oversamplesimple
X_con = pd.concat([train_X, train_y], axis=1) 
not_fraud = X_con[X_con.FraudResult==0]
fraud = X_con[X_con.FraudResult==1]
fraud_upsampled = resample(fraud, replace=True, n_samples=len(not_fraud), # match number in majority class
                          random_state=1) # reproducible results
upsampled = pd.concat([not_fraud, fraud_upsampled])
train_y_over_sampled = upsampled.FraudResult
train_X_over_sampled = upsampled.drop('FraudResult', axis=1)
upsampledmodel = RandomForestClassifier(n_estimators=36, criterion = 'entropy', random_state=1)
upsampledmodel.fit(train_X_over_sampled, train_y_over_sampled)
upsampled_pred = upsampledmodel.predict(val_X)
model_test = upsampledmodel.predict(OH_X_test)
description = descriptiontous+"upsampled, n_estimators=36, criterion = entropy"
metrics = listmetrics(val_y, upsampled_pred, "RandomForestClassifierUpperSampling", description)
m4Public = [0.678571428]
m4Private = [0.637168141]
res.append(metrics)
df = getscoreforcsv(index_val, model_test, name_file = "RandomForestClassifierUpSample.csv")

#Undersampling
not_fraud_downsampled  = resample(not_fraud, replace=False, n_samples=len(fraud), # match number in minority  class
                          random_state=1) # reproducible results
downsampled  = pd.concat([not_fraud_downsampled , fraud])
train_y_undersampled = downsampled.FraudResult
train_X_undersampled = downsampled.drop('FraudResult', axis=1)
undersampled = RandomForestClassifier(n_estimators=36, criterion = 'entropy', random_state=1)
undersampled.fit(train_X_over_sampled, train_y_over_sampled)
undersampled_pred = undersampled.predict(val_X)
model_test = undersampled.predict(OH_X_test)
description = descriptiontous+"undersampled, n_estimators=36, criterion = entropy"
metrics = listmetrics(val_y, undersampled_pred, "RandomForestClassifierUnderSampling", description)
m5Public = [0.549019607]
m5Private = [0.480769230]
res.append(metrics)
df = getscoreforcsv(index_val, model_test, name_file = "RandomForestClassifierUndersampled.csv")


dfres = listmetricsintodf(res)
dfres["OldPublicScore"] = [m1Public, m2Public, m3Public, m4Public, m5Public]
dfres["OldPrivateScore"] = [m1Private, m2Private, m3Private, m4Private, m5Private]
dfres["MeanOurMetrics"] = dfres[['Precision', 'Recall','F1-score','Mcc']].mean(axis=1)
dfres.to_csv('output/resultatsTestEverything.csv', mode='a',index=False,header = False) 
dfres

done


C:\Users\sarah\Documents\ECAM_non_One_Drive\Interligence artificiel\Projet Niko\AI5L_AILab_FraudDetection\EvaluationMetric.py:8: RuntimeWarning: invalid value encountered in scalar divide
  metric = tp / (tp+fp)
C:\Users\sarah\Documents\ECAM_non_One_Drive\Interligence artificiel\Projet Niko\AI5L_AILab_FraudDetection\EvaluationMetric.py:20: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp / (tp+fp)
C:\Users\sarah\Documents\ECAM_non_One_Drive\Interligence artificiel\Projet Niko\AI5L_AILab_FraudDetection\EvaluationMetric.py:8: RuntimeWarning: invalid value encountered in scalar divide
  metric = tp / (tp+fp)
C:\Users\sarah\Documents\ECAM_non_One_Drive\Interligence artificiel\Projet Niko\AI5L_AILab_FraudDetection\EvaluationMetric.py:20: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp / (tp+fp)


done
done
done
done


,Model,Description,Date,Precision,Recall,F1-score,LogLoss,Mcc,PublicScore,PrivateScore,OldPublicScore,OldPrivateScore,MeanOurMetrics
0,Decision Tree Classifier,Sans mediumMI MImax_leaf_nodes : 6 + with date columns,2023-05-06 20:51:29.534162,NaN,0.000,NaN,0.060284,0.000000,NaN,NaN,[0.666666666],[0.661016949],0.000000
1,Random Forest Classifier,Sans mediumMI MIn_estimators : 36 + Entropy,2023-05-06 20:51:29.969941,NaN,0.000,NaN,0.060284,0.000000,NaN,NaN,[0.69090909],[0.649122807],0.000000
2,XGBClassifier,Sans mediumMI MIscale_pos_weight = 22,2023-05-06 20:51:30.439168,0.152174,0.175,0.162791,0.108511,0.161685,NaN,NaN,[0.677419354],[0.676923076],0.162913
3,RandomForestClassifierUpperSampling,"Sans mediumMI MIupsampled, n_estimators=36, criterion = entropy",2023-05-06 20:51:31.381130,0.010616,0.875,0.020977,4.923675,0.087518,NaN,NaN,[0.678571428],[0.637168141],0.248528
4,RandomForestClassifierUnderSampling,"Sans mediumMI MIundersampled, n_estimators=36, criterion = entropy",2023-05-06 20:51:32.149516,0.010616,0.875,0.020977,4.923675,0.087518,NaN,NaN,[0.549019607],[0.48076923],0.248528


In [18]:
dfresult  = pd.read_csv("output/resultatsTestEverything.csv")
dfresult

,Model,Description,Date,Precision,Recall,F1-score,LogLoss,Mcc,PublicScore,PrivateScore,OldPublicScore,OldPrivateScore,MeanOurMetrics
0,Decision Tree Classifier,La basemax_leaf_nodes : 6 + with date columns,2023-05-06 20:37:01.701978,0.891892,0.825,0.857143,0.016578,0.857566,NaN,NaN,[0.666666666],[0.661016949],0.857900
1,Random Forest Classifier,La basen_estimators : 36 + Entropy,2023-05-06 20:37:02.891559,0.800000,0.900,0.847059,0.019592,0.848262,NaN,NaN,[0.69090909],[0.649122807],0.848830
2,XGBClassifier,La basescale_pos_weight = 22,2023-05-06 20:37:04.316049,0.840909,0.925,0.880952,0.015071,0.881747,NaN,NaN,[0.677419354],[0.676923076],0.882152
3,RandomForestClassifierUpperSampling,"La baseupsampled, n_estimators=36, criterion = entropy",2023-05-06 20:37:06.774426,0.765957,0.900,0.827586,0.022606,0.829975,NaN,NaN,[0.678571428],[0.637168141],0.830880
4,RandomForestClassifierUnderSampling,"La baseundersampled, n_estimators=36, criterion = entropy",2023-05-06 20:37:09.025865,0.765957,0.900,0.827586,0.022606,0.829975,NaN,NaN,[0.549019607],[0.48076923],0.830880
5,Decision Tree Classifier,Sans veryLow MImax_leaf_nodes : 6 + with date columns,2023-05-06 20:51:15.706650,0.942857,0.825,0.880000,0.013564,0.881780,NaN,NaN,[0.666666666],[0.661016949],0.882409
6,Random Forest Classifier,Sans veryLow MIn_estimators : 36 + Entropy,2023-05-06 20:51:16.916077,0.755556,0.850,0.800000,0.025621,0.801037,NaN,NaN,[0.69090909],[0.649122807],0.801648
7,XGBClassifier,Sans veryLow MIscale_pos_weight = 22,2023-05-06 20:51:18.180099,0.860465,0.925,0.891566,0.013564,0.891963,NaN,NaN,[0.677419354],[0.676923076],0.892249
8,RandomForestClassifierUpperSampling,"Sans veryLow MIupsampled, n_estimators=36, criterion = entropy",2023-05-06 20:51:20.658177,0.782609,0.900,0.837209,0.021099,0.838969,NaN,NaN,[0.678571428],[0.637168141],0.839697
9,RandomForestClassifierUnderSampling,"Sans veryLow MIundersampled, n_estimators=36, criterion = entropy",2023-05-06 20:51:22.937609,0.782609,0.900,0.837209,0.021099,0.838969,NaN,NaN,[0.549019607],[0.48076923],0.839697


In [36]:
print("Les meilleurs résultat")
dfresult.groupby(['Model'])['Precision','Recall','F1-score','LogLoss','MeanOurMetrics'].max()

Les meilleurs résultat


C:\Users\sarah\AppData\Local\Temp\ipykernel_18636\2367281751.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dfresult.groupby(['Model'])['Precision','Recall','F1-score','LogLoss','MeanOurMetrics'].max()


,Precision,Recall,F1-score,LogLoss,MeanOurMetrics
Model,,,,,
Decision Tree Classifier,0.942857,0.825,0.880000,0.060284,0.882409
Random Forest Classifier,0.800000,0.900,0.847059,0.060284,0.848830
RandomForestClassifierUnderSampling,0.782609,0.900,0.837209,4.923675,0.839697
RandomForestClassifierUpperSampling,0.782609,0.900,0.837209,4.923675,0.839697
XGBClassifier,0.860465,0.925,0.891566,0.108511,0.892249


In [33]:
print("Les pire résultat")
dfresult.groupby(['Model'])['Precision','Recall','F1-score','LogLoss','MeanOurMetrics'].min()

Les pire résultat


C:\Users\sarah\AppData\Local\Temp\ipykernel_18636\2101499374.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dfresult.groupby(['Model'])['Precision','Recall','F1-score','LogLoss','MeanOurMetrics'].min()


,Precision,Recall,F1-score,LogLoss,MeanOurMetrics
Model,,,,,
Decision Tree Classifier,0.500000,0.000,0.090909,0.013564,0.000000
Random Forest Classifier,0.566667,0.000,0.485714,0.019592,0.000000
RandomForestClassifierUnderSampling,0.010616,0.600,0.020977,0.021099,0.248528
RandomForestClassifierUpperSampling,0.010616,0.600,0.020977,0.021099,0.248528
XGBClassifier,0.152174,0.175,0.162791,0.013564,0.162913
